In [ ]:
from core.utils import Tibanna
from core import ff_utils


# for a given experiment set and some parameters like instrument
# print set of files and their partA hic workflow status
# if there are one that are running report the number of running cases
# if there are file pairs that don't have a corresponding part A, report them separately

env = 'fourfront-webdev'
tibanna = Tibanna(env=env)


ff = ff_utils.fdn_connection(key=tibanna.ff_keys)
files_resp = ff_utils.get_metadata('files', connection=ff)['@graph']


no_size = 0
for a_file in files_resp:
    print a_file
    break
 

'''
    # print 'Experiment', exp
    exp_resp = ff_utils.get_metadata(exp, connection=ff)
    exp_files = exp_resp['files']
    enzyme = exp_resp['digestion_enzyme']
    enzymes.append(enzyme)
    for fastq_file in exp_files[:]:
        file_resp = ff_utils.get_metadata(fastq_file, connection=ff, frame='embedded')      
        #if no uploaded file in the file item report and skip
        if not file_resp.get('filename'):
            print file_resp['accession'], "does not have a file"
            continue
        # check if file is in s3
        if not tibanna.s3.does_key_exist(file_resp['upload_key'], tibanna.s3.raw_file_bucket):
            print file_resp['accession'], "does not have a file in S3"
            continue
        # skip pair no 2
        if file_resp.get('paired_end')=='2':
            continue

        file_info = summarize_file(file_resp)

        # check for miseq
        if exclude_miseq:
            if file_info['sequencer'] == 'Illumina MiSeq':
                continue
        paired_file = file_info['paired_file']
        pair_file_resp = ff_utils.get_metadata(paired_file, connection=ff, frame='embedded')
        pair_file_info = summarize_file(pair_file_resp)

        # check consistency of paired file info
        # status differences gives error but there are multiple statuses that indicate complete
        # TODO fix it
        pairs_inconsistent = ""
        check_items = [ i for i in file_info.keys() if i not in ['file', 'paired_file', 'pair_no', 'alias']]
        for check_item in check_items:
            try:
                assert file_info[check_item] == pair_file_info[check_item]
            except AssertionError:
                print check_item, "not the same between pair", fastq_file, 'and', paired_file
                pairs_inconsistent += check_item + ', '
        wf_check = ''
        # check if md5 and qc are okay
        for info in [file_info, pair_file_info]:
            if (info['md5_status'] in status_done and info['file_status'] == 'uploaded' and
                info['fastqc_status'] in status_done and info['qc'] == True):
                wf_check += '+'
        rep = {"consistency": pairs_inconsistent,  
               "file1": file_info['alias'],
               "file2": pair_file_info['alias'],
               "const_check": pairs_inconsistent,
               "wf_check": wf_check, 
               "partA_wf": file_info['last_part_A'], 
               "partA_status": file_info['last_part_A_status']
               }
        # status differences gives error but there are multiple statuses that indicate complete
        # TODO fix it
        report.append(rep)
        if rep.get('const_check'):
            print rep['const_check']

    
# TODO need to add failed ones
# 1 completed pairs
pairs_completed = [i for i in report if i['partA_status']=='complete']
# 2 running pairs
pairs_running = [i for i in report if i['partA_status'] not in ['complete','did_not_run']]
# 3 no run pairs
pairs_did_not_run = [i for i in report if i['partA_status']=='did_not_run']
# 3a no run pairs with fine qc md5
pairs_ready_to_run = [(i['file1'], i['file2']) for i in pairs_did_not_run if i['wf_check'] == '++']
# 3b no run pairs with problematic qc md5
pairs_qcmd_problem = [(i['file1'], i['file2']) for i in pairs_did_not_run if i['wf_check'] != '++']

# 2 running pairs to run again
rerun_running_pairs = [(i['file1'], i['file2']) for i in report if i['partA_status'] not in ['complete','did_not_run']]

'''

